In [ ]:
%store -r knn_package_arn
%store -r ll_package_arn

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from sagemaker.predictor import Predictor


session = sagemaker.Session()
role = get_execution_role()

model = ModelPackage(
    role=role,
    model_package_arn=knn_package_arn,
    sagemaker_session=session
)

model.predictor_cls = Predictor

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = model.deploy(
    instance_type='ml.m5.xlarge', 
    initial_instance_count=1,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

In [ ]:
payload = {
    'instances': [
        {
          "features": [ 1.5, 2 ]
        },
    ]
}

predictor.predict(data=payload)

In [ ]:
def process_prediction_result(raw_result):
    first = raw_result['predictions'][0]
    return first['predicted_label']

In [ ]:
def predict(x, y, predictor=predictor):
    payload = {
        'instances': [
            {
              "features": [ x, y ]
            },
        ]
    }
    
    raw_result = predictor.predict(
        data=payload
    )
    
    return process_prediction_result(raw_result)

In [ ]:
predict(x=3, y=4)

In [ ]:
from time import sleep

def test_different_values(predictor=predictor):
    for x in range(-3, 3+1):
        for y in range(-3, 3+1):
            label = predict(x=x, y=y, predictor=predictor)
            print(f"x={x}, y={y}, label={label}")
            sleep(0.2)

In [ ]:
test_different_values()

In [ ]:
import boto3
client = boto3.client(service_name="sagemaker")


def create_model(model_package_arn, model_name, role=role, client=client):
    container_list = [
        {'ModelPackageName': model_package_arn}
    ]

    response = client.create_model(
        ModelName = model_name,
        ExecutionRoleArn = role,
        Containers = container_list
    )
    
    return response["ModelArn"]

In [ ]:
import string 
import random

def generate_random_string():
    return ''.join(
        random.sample(
        string.ascii_uppercase,12)
    )


model_name = f"ll-{generate_random_string()}"

model_arn = create_model(
    model_package_arn=ll_package_arn,
    model_name=model_name
)

In [ ]:
def create_endpoint_config(model_name, config_name, client=client):
    response = client.create_endpoint_config(
        EndpointConfigName = config_name,
        ProductionVariants=[{
            'InstanceType': "ml.m5.xlarge",
            'InitialInstanceCount': 1,
            'InitialVariantWeight': 1,
            'ModelName': model_name,
            'VariantName': 'AllTraffic'
        }]
    )

    return response["EndpointConfigArn"]

In [ ]:
config_name = f"config-{generate_random_string()}"

config_arn = create_endpoint_config(
    model_name=model_name,
    config_name=config_name
)

In [ ]:
response = client.update_endpoint(
    EndpointName=predictor.endpoint_name,
    EndpointConfigName=config_name
)

In [ ]:
print('Wait for update operation to complete')
sleep(60*5)

In [ ]:
predictor = Predictor(
    endpoint_name=predictor.endpoint_name,
    sagemaker_session=session,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

In [ ]:
payload = {
    'instances': [
        {
          "features": [ 1.5, 2 ]
        },
    ]
}

predictor.predict(data=payload)

In [ ]:
test_different_values(predictor=predictor)

In [ ]:
# predictor.delete_endpoint()

In [ ]:
endpoint_name = predictor.endpoint_name
%store endpoint_name